<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/Mnist_fs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare dataset


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D

In [2]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train/255.0
x_test=x_test/255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_train=x_train[...,tf.newaxis].astype("float32")
x_test=x_test[...,tf.newaxis].astype("float32")
x_train.shape

(60000, 28, 28, 1)

In [5]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(32)
test_data=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

## Define Model


In [6]:
class MyModel(tf.keras.Model):
  def __init__(self,number_of_classes):
    super().__init__()
    self.conv1=Conv2D(32,(3,3),activation='relu')
    self.flatten=Flatten()
    self.fc1=Dense(128,activation='relu')
    self.fc2=Dense(number_of_classes)
  
  def call(self,x):
    y=self.conv1(x)
    w=self.flatten(y)
    z=self.fc1(w)
    out=self.fc2(z)
    return out

model=MyModel(10)



In [7]:
loss_function=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer=tf.keras.optimizers.Adam()

In [8]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_acc=tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_acc=tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

In [15]:
def train_step(images,y):
  with tf.GradientTape() as tape:
    y_pred=model(images,training=True)
    loss=loss_function(y,y_pred)

  gradients=tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_acc(y,y_pred)

In [14]:
def test_step(images,y):
  y_pred=model(images,training=False)
  loss=loss_function(y,y_pred)
  test_loss(loss)
  test_acc(y,y_pred)

In [19]:
from tqdm import tqdm
def train():
  epochs=5
  for epoch in range(epochs):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()
    for images,labels in tqdm(train_data):
      train_step(images,labels)
    for images,labels in tqdm(test_data):
      test_step(images,labels)
    print('epoch: ',epoch+1)
    print('loss: ',train_loss.result())
    print('accuracy: ',train_acc.result())
    print('val loss: ',test_loss.result())
    print('val accuracy: ',test_acc.result())


In [20]:
train()

100%|██████████| 313/313 [00:02<00:00, 121.98it/s]


epoch:  1
loss:  tf.Tensor(0.0036108305, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.99901664, shape=(), dtype=float32)
val loss:  tf.Tensor(0.097044356, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9834, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 122.27it/s]


epoch:  2
loss:  tf.Tensor(0.0011848571, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.9996333, shape=(), dtype=float32)
val loss:  tf.Tensor(0.10097188, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9846, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 123.52it/s]


epoch:  3
loss:  tf.Tensor(0.0028531482, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.99913335, shape=(), dtype=float32)
val loss:  tf.Tensor(0.09623917, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9851, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 122.10it/s]


epoch:  4
loss:  tf.Tensor(0.0016837051, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.99936664, shape=(), dtype=float32)
val loss:  tf.Tensor(0.11682846, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9844, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 121.54it/s]


epoch:  5
loss:  tf.Tensor(0.0013638069, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.99951667, shape=(), dtype=float32)
val loss:  tf.Tensor(0.122738786, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9852, shape=(), dtype=float32)
